In [12]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from collections import OrderedDict, defaultdict
import pymongo


In [13]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.project2_db
collection = db.

In [30]:
jobs = []
companies = []
salaries = []
locations = []
links = []

url = 'https://www.indeed.com/jobs?q=data+science&l=California'

# Retrieve page with the requests module
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
results = soup.find_all('div',class_='row')

# Loop through results to retrieve title, company, location, salary, and link
for result in results:

    #job title
    try:
        job_title = result.a.text.strip()
    except:
        job_title = 'empty'
    jobs.append(job_title)

    #company
    try:
        company = result.find(name='span', attrs={'class':'company'}).text.strip()
    except:
        company = 'empty'
    companies.append(company)

    #location
    try:
        location = result.find(name='div', class_='location').text.strip() 
    except:
        try:
            location = result.find(name='span', class_='location').text.strip()
        except:
            location = 'no location'

    locations.append(location)

    #salary
    try:
        salary = result.find(name='span', class_='salary no-wrap').text.strip()
    except:
        salary = 'no salary info'

    salaries.append(salary)

    try:
        targets = result.find('a', class_='turnstileLink')
        link = 'https://www.indeed.com' + targets.attrs['href']
    except:
        link = 'empty'
    links.append(link)

In [31]:
print(jobs)

['R&D Product Application Scientist', 'Data Scientist / Analyst', 'Data Engineer', 'Business Intelligence Data Analyst', 'Senior Big Data Engineer', 'Data Science Intern Burbank, CA Summer 2019', 'Data Scientist', 'Data Scientist (entry level)', 'Analyst I, Data Science', 'Undergraduate Intern - Data Science', 'Data Scientist', 'BIOVIA - Data Science Internship, June - August 2019', 'Data Science Intern', 'Data Science Specialist', 'Summer 2019 Intern - Data Science', 'Data Scientist, Machine Learning Engineer', 'Data Scientist']


In [47]:
#Create dictionary to hold all information
posts = {
    'title': jobs,
    'location':locations,
    'company': companies,
    'url':links
    }

#convert dictionary to Pandas Data Frame
df = pd.DataFrame.from_dict(posts)

#drop duplicates of title and location are the same
df = df.drop_duplicates(subset=['title','location','company'],keep='last')

#convert data_frame back to dictionary
posts = df.to_dict('records')

In [48]:
posts

[{'title': 'R&D Product Application Scientist',
  'location': 'San Clemente, CA 92672',
  'company': 'Custom Flavors',
  'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B-Idbg_iRzxYCuduFuYmm2go2ku-cJi4rWPQyALxo-q929VtTjRfU_nOECacqEm-cDr3IMCz5xa0zpFqwZzCcOkuAsTuan5CEuiOHF-8y3lGsEtww2QT-ftxQpOicOx-mEGotEsl2rgz2d4_-HtpueHHoMnsShuKPmKKky2UMVRFp_VIBYD5lXt058iLYA7co3Es3pU78XsSPBBdmXXelIT7vAxEe3HwZoSgUTvUOJdb9thGlc__wOxYdUu5PFJJEI2Tc-wtecKjjX1KWvv6LMYZ24RYL9l7XaB7_so2E6PCcHXa3x6856JwmIQ8xNy_9YugTgTGgz4zXPo1ixr0AsPeHNBJL7PtLHpOqrxStdBlIqDyrEQHLIOCFXAP1CkoaqBvIYCqOjIwTi5CSz7FR46FZsl0cuSCFAeRBrgQWixkgv9jQozz4o5VtzqcbZX8A=&vjs=3&p=1&sk=&fvj=1'},
 {'title': 'Data Scientist / Analyst',
  'location': 'San Francisco Bay Area, CA',
  'company': 'BIS Consulting, Inc, on behalf of large enterprise Client',
  'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B9TYTfsA13ghXN5GR1FMS70VFxifqAIuagaQAmYtSjRAAD_kecdZD5T1laDI5PVDcQgUgyX7NdsxN7hMQP1ZqUF-whXACYsQfS1HoRsq8IzFih6v8koDbUr7j9YSz-

In [49]:
# Insert dictionary into MongoDB as a document
for post in posts:
    collection.insert_one(post)